In [1]:
!pip install BeautifulSoup4

In [2]:
!pip install requests

In [3]:
!pip install lxml

In [4]:
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html.parser')

#print(table)

table = soup.table
header = table.find_all('th')
#column header list = headList
header_list = [i.text for i in header]
#print(header_list)

data_list = []
row =[]
for i in table.select('tr')[1:]:
    for j in i.select('td'):
        row.append(j.text)
    data_list.append(row)
    row=[]

df = pd.DataFrame(data = data_list, columns=header_list)
df.head()
df.shape

(180, 3)

In [5]:
list = df.values.tolist()
list[0:5]

[['M1A\n', 'Not assigned\n', '\n'],
 ['M2A\n', 'Not assigned\n', '\n'],
 ['M3A\n', 'North York\n', 'Parkwoods\n'],
 ['M4A\n', 'North York\n', 'Victoria Village\n'],
 ['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n']]

In [6]:
list_new = np.array(list)
convert_list = np.char.strip(list_new , "\n")

In [7]:
data = pd.DataFrame(convert_list, columns={'Postal Code', 'Borough','Neighborhood'})
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
data.replace('', np.nan, inplace = True)
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Data Wrangling

In [9]:
d = data.values.tolist()
dN= []
i = 0 
N = len(data['Borough'])
for i in range(N):
    if d[i][1] == 'Not assigned':
        i = i +1 
    elif d[i][1] != 'Not assigned' and d[i][2] == 'NaN':
        dN.append([d[i][0],d[i][1],d[i][1]])
        i = i + 1
    else: 
        dN.append(d[i])
        i = i +1 

In [10]:
#convert into DataFrame 
# DataN is the Dataframe name of the clean file which satisfy the following condition:
# *Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
DataN = pd.DataFrame(dN, columns ={'Postal Code', 'Borough', 'Neighborhood'})
data1 = DataN.sort_values(by=['Postal Code'], ascending = True)
data1.head()

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [11]:
#importing the geospatial coordinate in csv
filename = 'Geospatial_Coordinates (2).csv'
geo = pd.read_csv(filename)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#transform the dataframe DataN into an array dfL
dfL = data1.values.tolist()
#transfrom the dataframe geo into an array gfL
gfL = geo.values.tolist()
dfL[0][0]

'M1B'

In [13]:
postal = data1['Postal Code'].values.tolist()
data_final = []
i = 0
N = len(postal)
for i in range(N):
    if gfL[i][0] == postal[i]:
        data_final.append([postal[i], dfL[i][1], dfL[i][2], gfL[i][1], gfL[i][2]])
        i = i + 1 
    else:
        i = i + 1
        
data_final = pd.DataFrame(data_final)
data_final.head()

,0,1,2,3,4
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
data_final = data_final.rename(columns={0:'Postal Code',1:'Borough',2:'Neighborhood',3:'Latitude',4:'Longitude'})
data_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


## Create Map or Toronto Ontario

In [16]:
address = 'Toronto Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Ontario are 43.6534817, -79.3839347.


In [17]:
# create map of New York using latitude and longitude values
map = folium.Map(location = [latitude,longitude], zoom_start = 10)

# add markers to map
for postal, lat, lng, borough, neighborhood in zip(data_final['Postal Code'],data_final['Latitude'], data_final['Longitude'],data_final['Borough'], data_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough, postal)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

## Define Foursquare account

In [18]:
CLIENT_ID = '0TBPF53YTTY1VIMH2PCOZ0KUS1ZP5QBSX0WXX2ZJ05JWBLK5' # your Foursquare ID
CLIENT_SECRET = 'SABE2Q211TGTOPECWQUA4IJFAQLI1Q41QPS5QWG3N0PCTXUV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0TBPF53YTTY1VIMH2PCOZ0KUS1ZP5QBSX0WXX2ZJ05JWBLK5
CLIENT_SECRET:SABE2Q211TGTOPECWQUA4IJFAQLI1Q41QPS5QWG3N0PCTXUV


## I choose MB1 as my target spot of this project.

In [19]:
neighborhood_latitude = data_final.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = data_final.loc[0, 'Longitude'] # neighborhood longitude value

PostalCode_name = data_final.loc[0, 'Postal Code'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(PostalCode_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of M1B are 43.806686299999996, -79.19435340000001.


## Querying/Searching the list venue on 500meter radius at the M1B 

In [20]:

search_query = 'M1C'
radius=500
latitude = neighborhood_latitude
longitude = neighborhood_longitude
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=0TBPF53YTTY1VIMH2PCOZ0KUS1ZP5QBSX0WXX2ZJ05JWBLK5&client_secret=SABE2Q211TGTOPECWQUA4IJFAQLI1Q41QPS5QWG3N0PCTXUV&ll=43.806686299999996,-79.19435340000001&v=20180605&query=M1C&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb917100cc1fd001b5ac532'},
 'response': {'venues': [{'id': '4ee2f4e177c8e89316b5b677',
    'name': 'Cascades (Metro Waste)',
    'location': {'address': '45 Thornmount Dr',
     'lat': 43.807494027880885,
     'lng': -79.19507299669937,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.807494027880885,
       'lng': -79.19507299669937}],
     'distance': 106,
     'cc': 'CA',
     'city': 'Scarborough',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['45 Thornmount Dr', 'Scarborough ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d130941735',
      'name': 'Building',
      'pluralName': 'Buildings',
      'shortName': 'Building',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1589188491',
    'hasPerk': False},
   {'id': '5e6190f3badf1d0008f6aa92',
    'name': 'Metropolitan Heating & Air Condit

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues)
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.crossStreet,venuePage.id
0,4ee2f4e177c8e89316b5b677,Cascades (Metro Waste),"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",v-1589188491,False,45 Thornmount Dr,43.807494,-79.195073,"[{'label': 'display', 'lat': 43.80749402788088...",106,CA,Scarborough,ON,Canada,"[45 Thornmount Dr, Scarborough ON, Canada]",NaN,NaN,NaN
1,5e6190f3badf1d0008f6aa92,Metropolitan Heating & Air Conditioning,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",v-1589188491,False,20 Venture Dr,43.804875,-79.196832,"[{'label': 'display', 'lat': 43.8048752, 'lng'...",283,CA,Scarborough,ON,Canada,"[20 Venture Dr, Scarborough ON M1B 3R7, Canada]",M1B 3R7,NaN,NaN
2,4d534825bbf92d4350259c0f,Centum Optima Mortgages Inc. - Toronto,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1589188491,False,1345 Morningside Ave,43.807155,-79.201255,"[{'label': 'display', 'lat': 43.80715479950187...",556,CA,Toronto,ON,Canada,"[1345 Morningside Ave (at Mclevin Ave), Toront...",M1B 5K3,at Mclevin Ave,NaN
3,52e67f2a498ed98e81b4db5b,Pakeer Sahadevan - Re/MAX Community Realty Inc.,[],v-1589188491,False,1265 Morningside Avenue,43.802551,-79.199422,"[{'label': 'display', 'lat': 43.8025515, 'lng'...",614,CA,Toronto,ON,Canada,"[1265 Morningside Avenue, Toronto ON M1B 3V9, ...",M1B 3V9,NaN,505979460


In [24]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cascades (Metro Waste),Building,43.807494,-79.195073
1,Metropolitan Heating & Air Conditioning,Construction & Landscaping,43.804875,-79.196832
2,Centum Optima Mortgages Inc. - Toronto,Office,43.807155,-79.201255
3,Pakeer Sahadevan - Re/MAX Community Realty Inc.,None,43.802551,-79.199422


In [25]:
M1B.shape

NameError: name 'M1B' is not defined

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

##  Function to repeat the same process to all the neighborhoods in M1B

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# type your answer here

M1B_venues = getNearbyVenues(names=data_final['Neighborhood'],
                                   latitudes=data_final['Latitude'],
                                   longitudes=data_final['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [28]:
print(M1B_venues.shape)
M1B_venues.head()

(2115, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [29]:
M1B_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
Willowdale,40,40,40,40,40,40
Woburn,4,4,4,4,4,4
Woodbine Heights,9,9,9,9,9,9


In [30]:
print('There are {} uniques categories.'.format(len(M1B_venues['Venue Category'].unique())))

There are 266 uniques categories.


## Analyze Each Neighborhood

In [31]:
# one hot encoding
M1B_onehot = pd.get_dummies(M1B_venues[['Venue Category']], prefix="", prefix_sep="")
#removing/drop the column Neighborhood
M1B_onehot = M1B_onehot.drop(['Neighborhood'], axis =1 )

# add neighborhood column back to dataframe
df = M1B_venues['Neighborhood']

M1B_onehot = pd.concat([df,M1B_onehot], axis =1 )

In [32]:
M1B_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
a = M1B_onehot.columns
a.value_counts()

Gift Shop           1
Ramen Restaurant    1
Church              1
Record Shop         1
Bus Line            1
                   ..
Supplement Shop     1
Boutique            1
Farmers Market      1
Drugstore           1
Butcher             1
Length: 266, dtype: int64

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
M1B_grouped = M1B_onehot.groupby('Neighborhood').mean().reset_index()
M1B_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
M1B_grouped.shape

(95, 266)

## Let's print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in M1B_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = M1B_grouped[M1B_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4         Miscellaneous Shop  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.18
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3        Dance Studio  0.09
4  Athletics & Sports  0.09


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.11
1     Coffee Shop  0.11
2     Pizza Place  0.05
3  Ice Cream Shop  0.05
4   Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.09
1      Sandwich Place  0.09
2  Italian Restaurant  0.09
3         Cof

#### Let's put that into a *pandas* dataframe

In [37]:
## First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = M1B_grouped['Neighborhood']

for ind in np.arange(M1B_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(M1B_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Skating Rink,Athletics & Sports,Pool,Sandwich Place,Dance Studio,Coffee Shop,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Pharmacy,Ice Cream Shop,Sushi Restaurant,Deli / Bodega,Shopping Mall,Restaurant,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,Pharmacy,Indian Restaurant,Café,Pub,Sushi Restaurant,Liquor Store


##  Cluster Neighborhoods

In [160]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

M1B_grouped_clustering = M1B_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(M1B_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
knn = kmeans.labels_.astype(int)
knn = pd.DataFrame(knn)

In [161]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,1,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Skating Rink,Athletics & Sports,Pool,Sandwich Place,Dance Studio,Coffee Shop,Gym
2,1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Pharmacy,Ice Cream Shop,Sushi Restaurant,Deli / Bodega,Shopping Mall,Restaurant,Fried Chicken Joint
3,1,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,1,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,Pharmacy,Indian Restaurant,Café,Pub,Sushi Restaurant,Liquor Store


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [162]:
# add clustering labels

MB1_merged = data_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
MB1_merged = MB1_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


MB1_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,History Museum,Bar,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Bank,Mexican Restaurant,Yoga Studio,Distribution Center,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Korean Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Gas Station,Hakka Restaurant,Bakery,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Dessert Shop,Dim Sum Restaurant


In [171]:
MB1_merged = MB1_merged.dropna()

In [172]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

type(MB1_merged['Cluster Labels'])

Libraries imported.


pandas.core.series.Series

## Create Map

In [173]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(MB1_merged['Latitude'], MB1_merged['Longitude'], MB1_merged['Neighborhood'], MB1_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters